In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange, repeat
import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin
import torch.nn as nn
### fucking around
from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
from train import getSamples, getUpSamples, getSpacing

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
name = r'box_plant2'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
training_data = preProcess(data)
np.random.shuffle(training_data)

Preparing data ... (1200/1203)

In [5]:
num_points = 10240
sample_magnitude = training_data[:num_points,0]
sample_magnitude = rearrange(sample_magnitude, 'n -> n 1')
angles = training_data[:num_points,1:3]
origins = training_data[:num_points,3:6]

In [6]:
elev = angles[:,0]
pan = angles[:,1]
### Return angles back to directions
x_tilde, y_tilde, z_tilde = cos(elev)*cos(pan), cos(elev)*sin(pan), sin(elev)      
unit_vectors = np.vstack((x_tilde, y_tilde, z_tilde))
unit_vectors = rearrange(unit_vectors, 'a b -> b a')
pos = unit_vectors*sample_magnitude + origins

In [7]:
gt_dist_tensor = torch.tensor(ground_truth_distance)
ang_tensor = torch.tensor(angles)
origins_tensor = torch.tensor(origins)

NameError: name 'ground_truth_distance' is not defined

In [ ]:
sample_pos, sample_ang, sample_org = getSamples(origins_tensor, ang_tensor, gt_dist_tensor, num_bins=100)
upsample_pos, upsample_ang, upsample_gt_distance = getUpSamples(origins_tensor, ang_tensor, gt_dist_tensor, num_rolls=0)

In [ ]:
# tile distances
gt_distance_tiled = torch.tensor(repeat(ground_truth_distance, 'b -> (b n) 1', n=100))

# stack the upsampled position to sampled positions
pos = (torch.vstack((sample_pos, upsample_pos)))
ang = (torch.vstack((sample_ang, upsample_ang)))
gt_dis = (torch.vstack((gt_distance_tiled,upsample_gt_distance)))
org = (torch.vstack((sample_org, upsample_pos)))


In [ ]:
actual_value_sigmoided = (getTargetValues(pos, gt_dis, org)).to(dtype = torch.float32)

/tmp/ipykernel_45895/3882400413.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  temp = torch.tensor((sample_positions - origins)**2)


In [ ]:
# returns pytorch tensor of sigmoid of projected SDF
def getTargetValues(sample_positions, gt_distance, origins, num_bins=100):
    # calculate distance from sample_position
    temp = torch.tensor((sample_positions - origins)**2)
    pos_distance = torch.sqrt(torch.sum(temp, dim=1, keepdim=True))
    
    # find the "projected" value
    sigmoid = nn.Sigmoid()
    values = sigmoid(-(pos_distance - gt_distance))
    return values

In [54]:
def generate_unit_vectors(direction, focal_length, height, width, width_resolution = 1000, height_resolution = 1000):
    
    # Initialize the output array
    unit_vectors = np.zeros((1000000, 3))
    
    # Compute the pixel coordinates
    x_coords, y_coords = np.meshgrid(np.arange(height_resolution), np.arange(width_resolution))
    x_coords = x_coords * (width / width_resolution)
    y_coords = y_coords * (height / height_resolution)
    
    # Flatten the coordinates
    x_coords = x_coords.flatten()
    y_coords = y_coords.flatten()

    # Compute the direction vectors
    for i in range(height_resolution*width_resolution):
        x = (x_coords[i] - width / 2)
        y = (y_coords[i] - height / 2)
        z = focal_length
        vec = np.array([x, y, z])
        unit_vectors[i] = vec / np.linalg.norm(vec)
    return unit_vectors
    # return vec

In [57]:
pts = generate_unit_vectors(np.array([1,0,0]),focal_length= 1, height = 0.5, width = 0.5)


In [58]:
# Convert the numpy array to an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])